# BayesianSearchCV

1. Apa itu hyperparameter tuning?
  
   Proses mencari nilai hyperparameter terbaik untuk meningkatkan performa model pada data validasi

2. Jelaskan metode BayesSearchCV!

   Implementasi Bayesian Optimization untuk pencarian hyperparamete (pada paket scikit-optimize). Lebih sample-efficient dibanding grid/random search

3. Bagaimana cara kerja BayesSearchCV?
  
   Menggunakan surrogate model (mis. Gaussian Process) + acquisition function untuk memilih pengamatan selanjutnya

4. Apa kelebihan BayesSearchCV dibandingkan metode hyperparameter tuning lainnya?
  
   Efisien, cepat menemukan konfigurasi baik pada jumlah evaluasi terbatas, cocok saat evaluasi mahal
    



In [14]:
# Load dataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Memuat dataset Iris
X, y = load_iris(return_X_y=True)

# Membagi dataset menjadi train dan test (75% : 25%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.75, random_state=0, stratify=y
)

print("Train shape:", X_train.shape)
print("Test shape :", X_test.shape)

Train shape: (112, 4)
Test shape : (38, 4)


In [15]:
# Definisikan Model dan parameter yang akan dioptimasi (Ruang Hyperparameter)
from sklearn.ensemble import RandomForestClassifier
from skopt.space import Integer, Real, Categorical

# Inisialisasi model dasar
model = RandomForestClassifier(random_state=42)

# Definisikan ruang pencarian hyperparameter
search_spaces = {
    'n_estimators': Integer(50, 300),         # jumlah pohon (trees)
    'max_depth': Integer(2, 30),              # kedalaman maksimum setiap pohon
    'min_samples_split': Integer(2, 10),      # minimum sampel untuk membagi node
    'min_samples_leaf': Integer(1, 10),       # minimum sampel per daun
    'max_features': Categorical(['sqrt', 'log2', None])  # jumlah fitur untuk dicoba per split
}

print("Model dan ruang hyperparameter siap untuk dioptimasi!")

Model dan ruang hyperparameter siap untuk dioptimasi!


In [16]:
!pip install scikit-optimize

# Fungsi Optimisasi Menggunakan BayesianSearchCV

In [17]:
# Inisialisasi BayesSearchCV
from skopt import BayesSearchCV

# Membuat objek BayesSearchCV
bayes = BayesSearchCV(
    estimator=model,              # model yang akan dioptimasi
    search_spaces=search_spaces,  # ruang hyperparameter
    n_iter=25,                    # jumlah iterasi pencarian
    scoring='accuracy',           # metrik evaluasi
    cv=3,                         # jumlah fold cross-validation
    n_jobs=-1,                    # gunakan semua core CPU
    random_state=42,              # untuk hasil konsisten
    verbose=1                     # tampilkan progres
)

print("✅ BayesSearchCV berhasil diinisialisasi!")

✅ BayesSearchCV berhasil diinisialisasi!


In [18]:
# Jalankan optimisasi
# Melatih (fit) model dengan BayesSearchCV
bayes.fit(X_train, y_train)

# Menyimpan model terbaik
best_model = bayes.best_estimator_

# Menampilkan hasil tuning
print("\n✅ Proses optimisasi selesai!")
print("Best Parameters :", bayes.best_params_)
print("Best CV Score   :", bayes.best_score_)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fi

In [20]:
# Evaluasi Model
# Prediksi data test menggunakan model terbaik
y_pred = best_model.predict(X_test)

# Import necessary metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Tampilkan hasil evaluasi
print("\n=== HASIL EVALUASI MODEL ===")
print("Akurasi Test :", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


=== HASIL EVALUASI MODEL ===
Akurasi Test : 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38

Confusion Matrix:
 [[13  0  0]
 [ 0 13  0]
 [ 0  0 12]]
